In [42]:
import pickle
import os 
import pandas as pd
import numpy as np  
import re
import copy

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

from bs4 import BeautifulSoup

# idx = 10
# for id in [1, 10, 14, 15]:
#     pickle_file = open(f"../data/features/car_number_{id + 1}.pkl", "rb") ##### offset is 1
#     unpickled = pickle.load(pickle_file)

#     basic_info = unpickled[0]
#     basic_info.update(unpickled[1])
#     additional_info = unpickled[2]

#     if 'price' in basic_info:
#         print(id, basic_info['price'])
    # print(prices[id])
# print("=== BASIC INFO ===\n", basic_info.keys())
# print("\n=== ADDITIONAL INFO ===\n", additional_info)

# Preprocessing of the prices for all vehicles

In [43]:
prices_path = '../data/prices_for_all_vehicles.pkl'
with open(prices_path, 'rb') as pp:
    unpickled_prices = pickle.load(pp)
    unpickled_prices = list(map(lambda x: (x[0], int(x[1][:-2].replace('.', '')) if '€' in x[1] else -1), unpickled_prices))
    unpickled_prices = list(filter(lambda x: True if x[1] > 0 else False, unpickled_prices))
prices = dict(unpickled_prices)

## Collecting car features and unpickling them

In [65]:
def generate_price(basic_info, scale=1e6):
    """
    Used to generate price in the times when the original price was not yet scrapped.
    """
    mileage_str_raw = basic_info['mileageFromOdometer'] 
    mileage_str = re.sub("[^0-9]", "", mileage_str_raw[ : -4]) 
    # price = scale * 1 / (float(mileage_str) + 1)
    # price = float(mileage_str)
    price = scale - float(mileage_str)

    return price


def collect_raw_data(data_folder, prices):
    """
    Returns a list where each element is a dictionary corresponding to a single car.

    """
    
    raw_data = []
    file_names = os.listdir(data_folder)

    # for file_name in file_names:
    for car_idx in prices.keys():

        file_name = f"car_number_{car_idx + 1}.pkl" #+1 is for offset
        if file_name not in file_names:
            continue

        file_path = os.path.join(data_folder, file_name)
        pickle_file = open(file_path, "rb")
        unpickled = pickle.load(pickle_file) 
        try:
            basic_info = unpickled[0]
            basic_info.update(unpickled[1])
            additional_info = unpickled[2]
        except(TypeError):
            return raw_data


        # print(file_name)
        # print(f"{car_idx} -- {prices[car_idx]}\n\n")
        # print(file_path)
        # print(basic_info)

        # price = generate_price(basic_info)
        try:
            price = prices[car_idx]
            basic_info.update({'price' : price})
            raw_data.append(basic_info)
        except(KeyError):
            continue

    return raw_data

def print_dict(raw_data):
    # print(raw_data[0])
    for r in raw_data:
        print(r, ' ---- ', raw_data[r])
    print('Length =', len(raw_data))    

In [66]:
features_folder = "../data/features"

raw_data = collect_raw_data(features_folder, prices)

# list(prices.keys())
# prices

print_dict(raw_data)


../data/features/car_number_1.pkl  not working! dict_keys([0, 2, 3, 6, 8, 10, 14, 15, 20, 21, 23, 25, 30, 33, 35, 36, 44, 48, 52, 53, 61, 66, 69, 80, 82, 84, 95, 96, 101, 114, 116, 117, 123, 126, 128, 129, 134, 137, 138, 144, 151, 152, 159, 166, 168, 169, 170, 171, 175, 184, 193, 194, 195, 197, 199, 204, 221, 223, 231, 233, 241, 242, 244, 246, 247, 253, 254, 257, 258, 260, 262, 266, 273, 274, 275, 277, 278, 282, 283, 286, 288, 289, 293, 294, 296, 297, 298, 299, 300, 302, 303, 305, 314, 316, 318, 330, 332, 334, 335, 340, 344, 348, 350, 353, 355, 356, 358, 368, 369, 370, 375, 380, 381, 382, 383, 386, 388, 391, 394, 397, 399, 400, 401, 411, 413, 416, 417, 420, 426, 434, 440, 458, 460, 465, 467, 472, 484, 485, 486, 490, 495, 498, 505, 508, 514, 520, 526, 534, 541, 542, 543, 544, 551, 558, 562, 567, 570, 579, 583, 584, 585, 586, 593, 595, 597, 598, 607, 610, 613, 614, 616, 619, 620, 621, 623, 625, 629, 633, 634, 638, 639, 643, 646, 649, 653, 658, 659, 660, 662, 663, 665, 666, 667, 668, 670,

## Discarding most of the features and keeping only the interesting ones

In [28]:
def preprocess_features(data_raw, keys_to_preprocess=[]):

    data_clean = copy.copy(data_raw)
    
    if ('productionDate' in keys_to_preprocess):
        data_clean['productionDate'] = int(data_raw['productionDate'])

    if ('vehicleEngine' in  keys_to_preprocess):
        data_clean['vehicleEngine'] = float(data_raw['vehicleEngine'][ : -5])

    if ('Broj sedišta' in keys_to_preprocess):
        data_clean['Broj sedišta'] = int(data_raw['Broj sedišta'].split(' ')[0])

    if ('mileageFromOdometer' in keys_to_preprocess):
        no_dots = re.sub("[^0-9]", "", data_raw['mileageFromOdometer'][ : -4])
        data_clean['mileageFromOdometer'] = float(no_dots)

    if ('url' in keys_to_preprocess):
        data_clean['url'] = data_raw['url'][46 : ]

    return data_clean

In [21]:
keys_to_keep = [
    'brand', 'productionDate', 'bodyType', 'fuelType', 
    'vehicleEngine', 'mileageFromOdometer', 'Broj sedišta', 
    'Oštećenje', 'Menjač', 'Klima', 'price'
]

keys_to_preprocess = ['productionDate', 'vehicleEngine', 'Broj sedišta', 'mileageFromOdometer']
kept_values = []

for i in range(len(raw_data)):
    
    curr_features_clean = preprocess_features(raw_data[i], keys_to_preprocess)
    curr_features = [
        curr_features_clean[key] for key in keys_to_keep
    ] 
    
    kept_values.append(curr_features)
    
# print(kept_values)
# print_dict(curr_features_clean)

df_raw = pd.DataFrame(kept_values, columns=keys_to_keep)
df_raw.head()

NameError: name 'raw_data' is not defined

## Map 

In [20]:
# =======================================================================
# == AC mapping ==

AC_map = {'Nema klimu' : 0, 'Manuelna klima' : 1, 'Automatska klima' : 2}

# =======================================================================
# == Brands mapping ==

brands_file = open('../data/brands.txt', 'r', encoding='UTF-8')
brands_raw_str = brands_file.readline()
brands_file.close()

brands_raw = brands_raw_str.split('value="')[2 : ]
brands_clean = [x.replace("</select>", "<option ")[ : -17].replace('">', "$") for x in brands_raw]

brands_map = {}
for i, brand in enumerate(brands_clean):
    key = brand.split('$')[1]
    brands_map.update({key: i})
    
# =======================================================================
# == Map bodyType  == 

body_map = {
    'Limuzina' : 0, 'Hečbek' : 1, 'Karavan' : 2, 'Kupe' : 3, 'Kabriolet/Roadster' : 4, 
    'Monovolumen (MiniVan)' : 5, 'Džip/SUV' : 6, 'Pickup' : 7
    }

# =======================================================================
# == Map fuelType ==

fuel_map = {
    'Benzin' : 0, 'Dizel' : 1, 'Metan CNG' : 2, 'Benzin + Gas (TNG)' : 3,
    'Benzin + Metan (CNG)' : 4, 'Električni pogon' : 5, 'Hibridni pogon' : 6
}

# =======================================================================
# == Map Oštećenje ==

damage_map = {
    'Nije oštećen' : 0, 'Oštećen - u voznom stanju' : 1,
    'Oštećen - nije u voznom stanju' : 2
    }

# =======================================================================
# == Map Menjač ==

control_map = {
    'Manuelni 4 brzine' : 0, 'Manuelni 5 brzina' : 1, 
    'Manuelni 6 brzina' : 2, 'Poluautomatski' : 3,
    'Automatski / poluautomatski' : 4, 'Automatski' : 5
}


to_replace = {
    "Klima" : AC_map, "brand" : brands_map, "bodyType" : body_map,
    "fuelType" : fuel_map, "Oštećenje" : damage_map, "Menjač" : control_map
    }

df_clean = df_raw.replace(to_replace)
df_clean.head()

NameError: name 'df_raw' is not defined

## Transform data to polynomial features

In [41]:
data_np = df_clean.to_numpy()
features_raw = data_np[ : , : -1]
targets = data_np[ : , -1]

# scaler = StandardScaler()
# features = scaler.fit_transform(features_raw)

features = features_raw

In [43]:
degree = 1
poly = PolynomialFeatures(degree)

features_poly = poly.fit_transform(features)

train_features, test_features, train_targets, test_targets = train_test_split(
    features_poly, targets, test_size=0.33 
)

array([[1.00000e+00, 2.80000e+01, 2.00600e+03, 1.00000e+00, 1.00000e+00,
        5.10000e+01, 1.79000e+05, 5.00000e+00, 0.00000e+00, 1.00000e+00,
        1.00000e+00],
       [1.00000e+00, 7.70000e+01, 2.02000e+03, 6.00000e+00, 0.00000e+00,
        1.03000e+02, 5.00000e+00, 5.00000e+00, 0.00000e+00, 2.00000e+00,
        2.00000e+00],
       [1.00000e+00, 5.70000e+01, 2.01600e+03, 3.00000e+00, 1.00000e+00,
        1.50000e+02, 1.97600e+05, 5.00000e+00, 0.00000e+00, 4.00000e+00,
        2.00000e+00],
       [1.00000e+00, 5.70000e+01, 2.01500e+03, 0.00000e+00, 1.00000e+00,
        8.00000e+01, 1.48000e+05, 4.00000e+00, 0.00000e+00, 5.00000e+00,
        2.00000e+00],
       [1.00000e+00, 6.80000e+01, 2.00900e+03, 0.00000e+00, 1.00000e+00,
        8.00000e+01, 1.88763e+05, 5.00000e+00, 0.00000e+00, 1.00000e+00,
        2.00000e+00],
       [1.00000e+00, 1.90000e+01, 2.00800e+03, 0.00000e+00, 1.00000e+00,
        1.00000e+02, 1.92000e+05, 5.00000e+00, 0.00000e+00, 2.00000e+00,
        2.0000

## Linear regression

In [51]:
# Train
linear_regressor = LinearRegression()
linear_regressor.fit(train_features, train_targets)

test_preds_lr = linear_regressor.predict(test_features)
rmse_lr = mean_squared_error(test_targets, test_preds_lr, squared=False)

print("LR RMSE -- ", rmse_lr)

for i in range(min(len(train_preds), 10)):
    print("PRED -- ", train_preds[i])
    print("REAL -- ", train_targets[i])
    print("\n\n")

# Test

PRED --  808000.0
REAL --  808000.0



PRED --  811237.0
REAL --  811237.0



PRED --  999995.0
REAL --  999995.0



PRED --  821000.0
REAL --  821000.0





## K Nearest Neighbors

In [52]:
df_clean

,brand,productionDate,bodyType,fuelType,vehicleEngine,mileageFromOdometer,Broj sedišta,Oštećenje,Menjač,Klima,price
0,28,2006,1,1,51.0,179000.0,5,0,1,1,821000.0
1,77,2020,6,0,103.0,5.0,5,0,2,2,999995.0
2,57,2016,3,1,150.0,197600.0,5,0,4,2,802400.0
3,57,2015,0,1,80.0,148000.0,4,0,5,2,852000.0
4,68,2009,0,1,80.0,188763.0,5,0,1,2,811237.0
5,19,2008,0,1,100.0,192000.0,5,0,2,2,808000.0


In [ ]:
N_neighbors = 4

# Train
knn = KNeighborsRegressor(n_neighbors=N_neighbors)
knn.fit(train_features, train_targets)

test_preds_knn = knn.predict(test_features)
rmse_knn = mean_squared_error(test_targets, test_preds_knn, squared=False)

print("KNN RMSE -- ", rmse_knn)

for i in range(0):
    print("PRED -- ", test_preds[i])
    print("REAL -- ", test_targets[i])
    print("\n\n")

# Test

KNN RMSE --  8321.482870971675
